In [1]:
from hdbcli import dbapi
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Connection database

In [2]:
connection = dbapi.connect('34.240.190.174', 39015, 'SYSTEM', 'Glorp2018!')
connection.isconnected()

True

In [3]:
cursor = connection.cursor()

# Teil 1

## 1. Ermitteln Sie die häufigsten Folgen von Adjektiv-Nomen Bigrammen im Corpus (Hinweis: wenn Sie mit SQL arbeiten, können Sie einen non-equi auto-join verwenden)

In [4]:
sql = 'select top 10 t1.TA_TOKEN, t2.TA_TOKEN, count(*) from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)
adjective_noun = cursor.fetchall()
adjective_noun_df = pd.DataFrame(adjective_noun)
adjective_noun_df.columns = ['adjective', 'noun', 'count']
adjective_noun_df

,adjective,noun,count
0,REAR,TIRE,4569
1,STEERING,WHEEL,3154
2,FRONT,TIRE,2901
3,APPROXIMATE,FAILURE,2412
4,SIDE,TIRE,2353
5,SAME,PROBLEM,2143
6,ORIGINAL,EQUIPMENT,2121
7,FRONT,END,1962
8,SEVERAL,TIMES,1887
9,NEW,TIRES,1861


## 2. Ermitteln Sie die häufigsten Co-occurrences von Adjektiven innerhalb eines Satzes (Position egal).

In [5]:
sql = 'select top 10 t1.TA_TOKEN as adjective, t2.TA_TOKEN as adjective2, count(*) from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER<t2.TA_COUNTER and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'adjective\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)
adj_adj = cursor.fetchall()
adj_adj_df = pd.DataFrame(adj_adj)
adj_adj_df.columns = ['adjective', 'adjective 2', 'count']
adj_adj_df

,adjective,adjective 2,count
0,RIGHT,REAR,1858
1,FRONT,SIDE,1752
2,REAR,SIDE,1641
3,LEFT,REAR,1627
4,RIGHT,FRONT,1496
5,OTHER,SAME,1440
6,LEFT,FRONT,1200
7,FRONT,FRONT,1131
8,REAR,REAR,1105
9,SAME,SAME,1053


## 3. Berechnen Sie die top-3 tf-idf (ntn nach der SMART Notation) Werte von Nomen im Corpus. Schauen Sie sich die entsprechenden Dokumente und die jeweiligen tf bzw. idf Wertkomponenten an.

### tf

In [ ]:
# create SQL-View MAX_FREQ_NOUN
sql = 'create view MAX_FREQ_NOUN as select top 1 t1.TA_TOKEN as noun, count(*) as maxfreq from "SYSTEM"."$TA_CDESCRIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-View MAX_FREQ_NOUN
sql_drop_view = 'drop view MAX_FREQ_NOUN'
cursor.execute(sql_drop_view)

In [6]:
# tf
sql = 'select top 3 t1.TA_TOKEN, (count(*)/t2.maxfreq) from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."MAX_FREQ_NOUN" as t2 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.maxfreq order by count(*) desc'
cursor.execute(sql)
tf = cursor.fetchall()
tf_df = pd.DataFrame(tf)
tf_df.columns = ['noun', 'tf']
tf_df

,noun,tf
0,VEHICLE,1
1,CAR,0.826885
2,TIRE,0.798524


### idf

In [ ]:
# create SQL-View MAX_COUNT_NOUNS
sql = 'create view MAX_COUNT_NOUNS as select top 3 t1.TA_TOKEN as noun from "SYSTEM"."$TA_CDESCRIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create SQL-View COUNT_DOCS
sql = 'create view COUNT_DOCS as select count(DISTINCT cmplid) as n_total from "SYSTEM"."$TA_CDESCRIND"'
cursor.execute(sql)

In [ ]:
# create SQL-View VALUES_IDF
sql = 'create view VALUES_IDF as select t1.TA_TOKEN, count(DISTINCT cmplid) as count_docs_term, t3.n_total from "SYSTEM"."$TA_CDESCRIND" as t1, MAX_COUNT_NOUNS as t2, COUNT_DOCS as t3 where TA_TOKEN=t2.noun group by t1.TA_TOKEN, t3.n_total'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view <view_name>'
cursor.execute(sql_drop_view)

In [7]:
# idf
sql = 'select TA_TOKEN, count_docs_term, n_total, LOG(10,(n_total/count_docs_term)) as idf from VALUES_IDF'
cursor.execute(sql)
idf = cursor.fetchall()
idf_df = pd.DataFrame(idf)
idf_df.columns = ['Token', 'count_docs_term', 'n_total', 'idf']
idf_df

,Token,count_docs_term,n_total,idf
0,TIRE,28150,102848,0.562717
1,VEHICLE,40399,102848,0.405825
2,CAR,27254,102848,0.576766


## 4. Berücksichtigen Sie nur Adjektiv-Nomen Bigramme in Dokumenten mit der ID<1000000. Ermitteln Sie anhand des Chi2 Tests die 3 statistisch signifikantesten und die 3 am wenigsten signifikant zusammenhängenden Bigramme mit w1=* (beliebig) und w2=Tire.

In [ ]:
# create view
# all adjective-noun bigrams with ID < 1000000 and w1=* and w2=Tire
sql = 'create view adjective_noun_bigram as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_w2 from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t1.cmplid<1000000 and t2.TA_TOKEN=\'TIRE\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create view
# all adjective-noun bigrams with ID < 1000000 and w1=* and w2=not_Tire
sql = 'create view adjective_noun_bigram_not_tire as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_not_w2 from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t1.cmplid<1000000 and t2.TA_TOKEN!=\'TIRE\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view <view_name>'
cursor.execute(sql_drop_view)

In [8]:
cursor.execute("""
select adjective, noun, o11, o12, o21, o22 from 
(
select t1.adjective, t1.noun, w1_and_w2 as o11, (sum_w1_and_w2-SUM(w1_and_w2)) as o12, t3.sum_w1_and_not_w2 as o21, (sum_NOT_w2 - t3.sum_w1_and_not_w2) as o22 from 
"SYSTEM"."ADJECTIVE_NOUN_BIGRAM" as t1, 
(select SUM(w1_and_w2) as sum_w1_and_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM") as t2,
(select adjective, SUM(w1_and_not_w2) as sum_w1_and_not_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TIRE" group by adjective) as t3,
(select SUM(w1_and_not_w2) as sum_NOT_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TIRE" order by sum_NOT_w2 desc) as t4 where t3.adjective=t1.adjective group by t1.adjective, t1.noun, w1_and_w2, t2.sum_w1_and_w2, t3.sum_w1_and_not_w2, t4.sum_NOT_w2
)
""");
data = cursor.fetchall();

In [9]:
values_list = []
header = ["adjective", "noun", "o11", "o12", "o21", "o22", "chi_square"];
for item in data:
    o11=item[2]
    o12=item[3]
    o21= item[4]
    o22=item[5]
    chi_square = ((102848 * (o11 * o22 - o12 * o21)**2) / ((o11 + o12) * (o11 + o21) * (o12 + o22) * (o21 + o22)))
    values_list.append((item[0], item[1], item[2], item[3], item[4], item[5], chi_square))

values_list.sort(key=lambda x: x[6], reverse=True);
df = pd.DataFrame(values_list)
df.columns = [header]
df

,adjective,noun,o11,o12,o21,o22,chi_square
0,REAR,TIRE,4206,13796,5806,143638,6660.536074
1,SPARE,TIRE,828,17174,63,149381,3872.408630
2,SIDE,TIRE,1673,16329,2049,147395,2849.785685
3,FRONT,TIRE,2590,15412,6153,143291,2103.259287
4,FLAT,TIRE,480,17522,569,148875,828.103007
...,...,...,...,...,...,...,...
523,READ,TIRE,1,18001,8,149436,0.000747
524,TOXIC,TIRE,1,18001,8,149436,0.000747
525,TWIN,TIRE,2,18000,17,149427,0.000614
526,BROUGHT,TIRE,5,17997,41,149403,0.000415


## 5. Implementieren Sie eine Ähnlichkeitsanfrage, die für einen gegebenen Anfragevektor (egal ob existierendes Dokument oder eigene Suchanfrage) die gerankten top n Dokumente anhand der Wortvektoren (nur Nomen ohne IDF-Gewichtung!) zurückgibt mit Hilfe von 
## a. Skalarprodukt (nnn nach SMART Notation)


In [10]:
def scalar_product(id):
    sql = 'SELECT CMPLID, sum(PROD) AS SCALARPRODUCT FROM (SELECT a.CMPLID, a.WORD, a.TERM * b.CountReq AS PROD FROM TERMS_NHTSA AS a, (SELECT TA_TOKEN, count(*) AS CountReq FROM "SYSTEM"."$TA_CDESCRIND" WHERE CMPLID = '+id+' GROUP BY TA_TOKEN) AS b WHERE a.WORD = b.TA_TOKEN) GROUP BY CMPLID ORDER BY SCALARPRODUCT desc LIMIT 10;'
    cursor.execute(sql)
    idf = cursor.fetchall()
    idf_df = pd.DataFrame(idf)
    print(idf_df)

In [ ]:
# create view TERMS_NHTSA
sql = 'create view TERMS_NHTSA as SELECT CMPLID, TA_TOKEN AS WORD, count(*) AS TERM, POWER(count(*),2) AS TERMSQR FROM "SYSTEM"."$TA_CDESCRIND" WHERE TA_TYPE=\'noun\' GROUP BY CMPLID, TA_TOKEN'
cursor.execute(sql)

In [ ]:
# drop SQL-View
sql_drop_view = 'drop view TERMS_NHTSA'
cursor.execute(sql_drop_view)

## b. Kosinus Ähnlichkeit (nnc nach SMART Notation)

In [11]:
def cosinus(id):
    sql = 'SELECT CMPLID, SCALARPRODUCT / (Cos * (SELECT SQRT(sum(coutsqr)) FROM (SELECT TA_TOKEN, POWER(count(*),2) AS coutsqr FROM "SYSTEM"."$TA_CDESCRIND" WHERE CMPLID = '+id+' AND TA_TYPE = \'noun\' GROUP BY TA_TOKEN))) AS COSINUS FROM COS_NHTSA ORDER BY COSINUS DESC LIMIT 10'
    cursor.execute(sql)
    idf = cursor.fetchall()
    idf_df = pd.DataFrame(idf)
    print(idf_df)

In [ ]:
# create view COS_NHTSA
sql = 'create view COS_NHTSA as SELECT x.CMPLID AS CMPLID, x.SCALARPRODUCT AS SCALARPRODUCT, y.Cos AS Cos From (SELECT CMPLID, sum(PROD) AS SCALARPRODUCT FROM (SELECT a.CMPLID, a.WORD, a.TERM * b.CountReq AS PROD FROM TERMS_NHTSA AS a, (SELECT TA_TOKEN, count(*) AS CountReq FROM "SYSTEM"."$TA_CDESCRIND" WHERE CMPLID = 119408 GROUP BY TA_TOKEN) AS b WHERE a.WORD = b.TA_TOKEN) GROUP BY CMPLID) AS x, (SELECT CMPLID, SQRT(SUM(TERMSQR)) AS Cos FROM TERMS_NHTSA GROUP BY CMPLID) AS y WHERE x.CMPLID = y.CMPLID'
cursor.execute(sql)

In [ ]:
# drop SQL-View
sql_drop_view = 'drop view COS_NHTSA'
cursor.execute(sql_drop_view)

## 6. Welches sind die jeweils ähnlichsten Dokumente für Complaint 119408? Wie unterscheiden sich die Ergebnisse grundsätzlich?

In [12]:
scalar_product('119408');

         0   1
0  1279795  37
1   765755  31
2   431818  28
3  1219041  27
4  1266333  26
5  1362301  26
6  1282926  26
7  1453481  25
8  1305599  25
9  1245452  24


In [13]:
cosinus('119408');

         0                                     1
0   119408                                     1
1   119409                                     1
2  1456180  0.8819171036881968635005385845464203
3  1426915  0.8728715609439695250643899416624782
4   442989  0.8641585652180931092236424027892403
5  1282926  0.8081708338082719302178786999304682
6  1209040   0.778289642116024569758635772967109
7  1283670  0.7647191129018725530240319269929149
8  1362301  0.7630574521488739735565932174873452
9   406176  0.7570332986102251781009008187750827


## 7.  Aufgaben 1-6 auf den eigenen Datensatz (T3N-Daten)

### 1. Ermitteln Sie die häufigsten Folgen von Adjektiv-Nomen Bigrammen im Corpus (Hinweis: wenn Sie mit SQL arbeiten, können Sie einen non-equi auto-join verwenden)

In [14]:
sql = 'select top 10 t1.TA_TOKEN, t2.TA_TOKEN, count(*) from "SYSTEM"."$TA_T3NTEXTIND" as t1, "SYSTEM"."$TA_T3NTEXTIND" as t2 where t1.ID=t2.ID and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)
adjective_noun = cursor.fetchall()
adjective_noun_df = pd.DataFrame(adjective_noun)
adjective_noun_df.columns = ['adjective', 'noun', 'count']
adjective_noun_df

,adjective,noun,count
0,vergangenen,Jahr,49
1,erster,Linie,48
2,ersten,Blick,39
3,Sublime,Text,34
4,letzten,Jahr,33
5,eigenen,Angaben,30
6,Virtual,Reality,29
7,vergangenen,Jahren,27
8,letzten,Jahren,26
9,neuen,Modelle,25


### 2. Ermitteln Sie die häufigsten Co-occurrences von Adjektiven innerhalb eines Satzes (Position egal).

In [15]:
sql = 'select top 10 t1.TA_TOKEN as adjective, t2.TA_TOKEN as adjective2, count(*) from "SYSTEM"."$TA_T3NTEXTIND" as t1, "SYSTEM"."$TA_T3NTEXTIND" as t2 where t1.ID=t2.ID and t1.TA_COUNTER<t2.TA_COUNTER and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'adjective\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)
adj_adj = cursor.fetchall()
adj_adj_df = pd.DataFrame(adj_adj)
adj_adj_df.columns = ['adjective', 'adjective 2', 'count']
adj_adj_df

,adjective,adjective 2,count
0,neue,neuen,39
1,neuen,neuen,38
2,neuen,neue,23
3,neue,neue,23
4,neue,ersten,16
5,ersten,ersten,15
6,eigenen,eigenen,14
7,ersten,neuen,14
8,weitere,neue,13
9,ersten,neue,12


#### Without SQL

In [ ]:
t3n_df = pd.read_csv('../praktikum2/t3n_data_cleaned.csv')
t3n_df

In [ ]:
t3n_adjectives = t3n_df[t3n_df['TA_TYPE']=='adjective']

In [ ]:
grouped_coocurrencies = t3n_adjectives.groupby(['ID','TA_SENTENCE'])['TA_TOKEN_LOW']

In [ ]:
coocurrencies = {}

#generate "matrix" with coocurrencies. we count the coocurrencies of each adverb per sentence. 
for name, group in grouped_coocurrencies:
    for index_i, token_i in group.iteritems():
        type(token_i)
        if token_i not in coocurrencies:
            coocurrencies[token_i] = {}
            
        for index_j, token_j in group.iteritems():
            if token_j not in coocurrencies[token_i]:
                coocurrencies[token_i][token_j] = 0
            else:
                coocurrencies[token_i][token_j]+=1 

In [ ]:
ordered_coocurrencies = {}

for coocurrency in coocurrencies:
    coocurrencies_for_this_token = coocurrencies[coocurrency]
    for coocurrency_for_this_token in coocurrencies_for_this_token:
        if(coocurrency != coocurrency_for_this_token):
            ordered_coocurrencies[coocurrency + '-' + coocurrency_for_this_token] = coocurrencies_for_this_token[coocurrency_for_this_token] 
            
ordered_coocurrencies = {k: v for k, v in sorted(ordered_coocurrencies.items(), key=lambda item: item[1],reverse=True)}        
    

In [ ]:
def drop_dublicates(ordered_coocurrencies):
    for occurency in ordered_coocurrencies.copy():
        split_occurency = occurency.split('-')
        dublicate_occurency = split_occurency[1] + '-' + split_occurency[0]
        if dublicate_occurency in ordered_coocurrencies and occurency in ordered_coocurrencies:
            ordered_coocurrencies.pop(dublicate_occurency)
    return ordered_coocurrencies

In [ ]:
drop_dublicates(ordered_coocurrencies)

### 3. Berechnen Sie die top-3 tf-idf (ntn nach der SMART Notation) Werte von Nomen im Corpus. Schauen Sie sich die entsprechenden Dokumente und die jeweiligen tf bzw. idf Wertkomponenten an.

### tf

In [ ]:
# create SQL-View MAX_FREQ_NOUN_T3N
sql = 'create view MAX_FREQ_NOUN_T3N as select top 1 t1.TA_TOKEN as noun, count(*) as maxfreq from "SYSTEM"."$TA_T3NTEXTIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-View MAX_FREQ_NOUN_T3N
sql_drop_view = 'drop view MAX_FREQ_NOUN_T3N'
cursor.execute(sql_drop_view)

In [16]:
# tf
sql = 'select top 3 t1.TA_TOKEN, (count(*)/t2.maxfreq) from "SYSTEM"."$TA_T3NTEXTIND" as t1, "SYSTEM"."MAX_FREQ_NOUN_T3N" as t2 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.maxfreq order by count(*) desc'
cursor.execute(sql)
tf = cursor.fetchall()
tf_df = pd.DataFrame(tf)
tf_df.columns = ['noun', 'tf']
tf_df

,noun,tf
0,Google,1
1,Apple,0.87931
2,Unternehmen,0.634482


### idf

In [ ]:
# create SQL-View MAX_COUNT_NOUNS_T3N
sql = 'create view MAX_COUNT_NOUNS_T3N as select top 3 t1.TA_TOKEN as noun from "SYSTEM"."$TA_T3NTEXTIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create SQL-View COUNT_DOCS_T3N
sql = 'create view COUNT_DOCS_T3N as select count(DISTINCT ID) as n_total from "SYSTEM"."$TA_T3NTEXTIND"'
cursor.execute(sql)

In [ ]:
# create SQL-View VALUES_IDF_T3N
sql = 'create view VALUES_IDF_T3N as select t1.TA_TOKEN, count(DISTINCT ID) as count_docs_term, t3.n_total from "SYSTEM"."$TA_T3NTEXTIND" as t1, MAX_COUNT_NOUNS_T3N as t2, COUNT_DOCS_T3N as t3 where TA_TOKEN=t2.noun group by t1.TA_TOKEN, t3.n_total'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view VALUES_IDF_T3N'
cursor.execute(sql_drop_view)

In [17]:
# idf
sql = 'select TA_TOKEN, count_docs_term, n_total, LOG(10,(n_total/count_docs_term)) as idf from VALUES_IDF_T3N'
cursor.execute(sql)
idf = cursor.fetchall()
idf_df = pd.DataFrame(idf)
idf_df.columns = ['Token', 'count_docs_term', 'n_total', 'idf']
idf_df

,Token,count_docs_term,n_total,idf
0,Google,311,994,0.504626
1,Apple,274,994,0.559636
2,Unternehmen,409,994,0.385663


### 4. Berücksichtigen Sie alle Adjektiv-Nomen Bigramme in Dokumenten. Ermitteln Sie anhand des Chi2 Tests die 3 statistisch signifikantesten und die 3 am wenigsten signifikant zusammenhängenden Bigramme mit w1=* (beliebig) und w2=Text.

In [ ]:
# create view
# all adjective-noun bigrams and w1=* and w2=Text
sql = 'create view adjective_noun_bigram_T3N as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_w2 from "SYSTEM"."$TA_T3NTEXTIND" as t1, "SYSTEM"."$TA_T3NTEXTIND" as t2 where t1.ID=t2.ID and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t2.TA_TOKEN=\'Text\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create view
# all adjective-noun bigrams and w1=* and w2=not_Text
sql = 'create view adjective_noun_bigram_not_text as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_not_w2 from "SYSTEM"."$TA_T3NTEXTIND" as t1, "SYSTEM"."$TA_T3NTEXTIND" as t2 where t1.ID=t2.ID and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t2.TA_TOKEN!=\'Text\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view adjective_noun_bigram_T3N'
cursor.execute(sql_drop_view)

In [18]:
cursor.execute("""
select adjective, noun, o11, o12, o21, o22 from 
(
select t1.adjective, t1.noun, w1_and_w2 as o11, (sum_w1_and_w2-SUM(w1_and_w2)) as o12, t3.sum_w1_and_not_w2 as o21, (sum_NOT_w2 - t3.sum_w1_and_not_w2) as o22 from 
"SYSTEM"."ADJECTIVE_NOUN_BIGRAM_T3N" as t1, 
(select SUM(w1_and_w2) as sum_w1_and_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_T3N") as t2,
(select adjective, SUM(w1_and_not_w2) as sum_w1_and_not_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TEXT" group by adjective) as t3,
(select SUM(w1_and_not_w2) as sum_NOT_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TEXT" order by sum_NOT_w2 desc) as t4 where t3.adjective=t1.adjective group by t1.adjective, t1.noun, w1_and_w2, t2.sum_w1_and_w2, t3.sum_w1_and_not_w2, t4.sum_NOT_w2
)
""");
data = cursor.fetchall();

In [19]:
values_list = []
header = ["adjective", "noun", "o11", "o12", "o21", "o22", "chi_square"];
for item in data:
    o11=item[2]
    o12=item[3]
    o21= item[4]
    o22=item[5]
    chi_square = ((102848 * (o11 * o22 - o12 * o21)**2) / ((o11 + o12) * (o11 + o21) * (o12 + o22) * (o21 + o22)))
    values_list.append((item[0], item[1], item[2], item[3], item[4], item[5], chi_square))

values_list.sort(key=lambda x: x[6], reverse=True);
df = pd.DataFrame(values_list)
df.columns = [header]
df

,adjective,noun,o11,o12,o21,o22,chi_square
0,prägnanten,Text,1,46,1,23507,1087.674939
1,eingegebenen,Text,1,46,1,23507,1087.674939
2,markierten,Text,1,46,2,23506,722.244913
3,kompletten,Text,1,46,16,23492,120.463557
4,normalen,Text,1,46,18,23490,106.903648
5,kurzen,Text,1,46,31,23477,60.130692
6,richtigen,Text,1,46,39,23469,46.493854
7,Smart,Text,1,46,41,23467,43.900362
8,folgenden,Text,1,46,46,23462,38.388556


### 5. Implementieren Sie eine Ähnlichkeitsanfrage, die für einen gegebenen Anfragevektor (egal ob existierendes Dokument oder eigene Suchanfrage) die gerankten top n Dokumente anhand der Wortvektoren (nur Nomen ohne IDF-Gewichtung!) zurückgibt mit Hilfe von 
#### a. Skalarprodukt (nnn nach SMART Notation)

In [20]:
def scalar_product(id):
    sql = 'SELECT ID, sum(PROD) AS SCALARPRODUCT FROM (SELECT a.ID, a.WORD, a.TERM * b.CountReq AS PROD FROM TERMS_T3N AS a, (SELECT TA_TOKEN, count(*) AS CountReq FROM "SYSTEM"."$TA_T3NTEXTIND" WHERE ID = \''+id+'\' GROUP BY TA_TOKEN) AS b WHERE a.WORD = b.TA_TOKEN) GROUP BY ID ORDER BY SCALARPRODUCT desc LIMIT 10;'
    cursor.execute(sql)
    idf = cursor.fetchall()
    idf_df = pd.DataFrame(idf)
    print(idf_df)

In [ ]:
# create view TERMS_T3N
sql = 'create view TERMS_T3N as SELECT ID, TA_TOKEN AS WORD, count(*) AS TERM, POWER(count(*),2) AS TERMSQR FROM "SYSTEM"."$TA_T3NTEXTIND" WHERE TA_TYPE=\'noun\' GROUP BY ID, TA_TOKEN'
cursor.execute(sql)

In [ ]:
# drop SQL-View
sql_drop_view = 'drop view TERMS_T3N'
cursor.execute(sql_drop_view)

#### b. Kosinus Ähnlichkeit (nnc nach SMART Notation)

In [21]:
def cosinus(id):
    # create view COS_T3N
    sql = 'create view COS_T3N as SELECT x.ID AS ID, x.SCALARPRODUCT AS SCALARPRODUCT, y.Cos AS Cos From (SELECT ID, sum(PROD) AS SCALARPRODUCT FROM (SELECT a.ID, a.WORD, a.TERM * b.CountReq AS PROD FROM TERMS_T3N AS a, (SELECT TA_TOKEN, count(*) AS CountReq FROM "SYSTEM"."$TA_T3NTEXTIND" WHERE ID = \''+id+'\' GROUP BY TA_TOKEN) AS b WHERE a.WORD = b.TA_TOKEN) GROUP BY ID) AS x, (SELECT ID, SQRT(SUM(TERMSQR)) AS Cos FROM TERMS_T3N GROUP BY ID) AS y WHERE x.ID = y.ID'
    cursor.execute(sql)
    
    sql = 'SELECT ID, SCALARPRODUCT / (Cos * (SELECT SQRT(sum(coutsqr)) FROM (SELECT TA_TOKEN, POWER(count(*),2) AS coutsqr FROM "SYSTEM"."$TA_T3NTEXTIND" WHERE ID = \''+id+'\' AND TA_TYPE = \'noun\' GROUP BY TA_TOKEN))) AS COSINUS FROM COS_T3N ORDER BY COSINUS DESC LIMIT 10'
    cursor.execute(sql)
    idf = cursor.fetchall()
    idf_df = pd.DataFrame(idf)
    
    # drop SQL-View
    sql_drop_view = 'drop view COS_T3N'
    cursor.execute(sql_drop_view)
    
    print(idf_df)

### 6. Welches sind die jeweils ähnlichsten Dokumente für das Dokument mit der ID 54ff8c69-007f-49a8-94cf-093fb5b0951b? Wie unterscheiden sich die Ergebnisse grundsätzlich?

In [22]:
scalar_product('54ff8c69-007f-49a8-94cf-093fb5b0951b')

                                      0   1
0  54ff8c69-007f-49a8-94cf-093fb5b0951b  91
1  53046188-b726-4c86-828a-617cc0a14894  73
2  29983671-fcf8-4c0d-a447-dc5a57384f02  70
3  21d5a31b-40e1-4e27-a9d4-b7227c967dd9  51
4  72f4cee7-056e-4b4b-905e-f795dc9bd082  47
5  d22af3b9-f335-466d-a895-4aba5bf472f8  45
6  15f17f43-2bcf-4213-8e23-007afdff9837  42
7  6713f823-292b-4974-a3f9-69a3a8960020  42
8  5ac9ac3a-5971-4d88-b281-9e7edd477301  32
9  0268456f-6e26-4462-a912-cea84acf1da9  31


In [23]:
cosinus('54ff8c69-007f-49a8-94cf-093fb5b0951b')

                                      0                                     1
0  54ff8c69-007f-49a8-94cf-093fb5b0951b   1.011111111111111111111111111111111
1  53046188-b726-4c86-828a-617cc0a14894  0.3008934481546112521263477148911646
2  29983671-fcf8-4c0d-a447-dc5a57384f02  0.2687145064945125899172116757437412
3  72f4cee7-056e-4b4b-905e-f795dc9bd082  0.2659572696738088829814135066188706
4  5ac9ac3a-5971-4d88-b281-9e7edd477301  0.2587046667054218384201668929238637
5  15f17f43-2bcf-4213-8e23-007afdff9837  0.2573251177328327119329988151470316
6  21d5a31b-40e1-4e27-a9d4-b7227c967dd9  0.2222649531566085695140507944958648
7  5d4d1547-8061-4328-a9e9-2866144623ea   0.200331400392989446283827163361484
8  e9f6594d-714b-4f35-942f-2205dbd2459e  0.1982936921576542815137590055694631
9  b0291f79-bc9b-4b5e-9410-5dce1076a7b4  0.1981473482419403759187452591453117


# Teil 2
## 1. Implementieren oder wrappen Sie eine Python-Funktion (gerne mit Hilfe üblicher Bibliotheken wie sklearn.metrics.precision_recall_curve), die für Ihre vorbereiteten Anfragevektoren und die mit der Kosinus-Ähnlichkeit ermittelten Dokumente die Precision-Recall Plots zeichnet (Interpolation ist nicht nötig).

In [24]:
def precisionRecall(param):
    gefunden_sql = 'SELECT distinct ID FROM "SYSTEM"."$TA_T3NTEXTIND" WHERE TA_TOKEN = ('+param+')'    
    cursor.execute(gefunden_sql)
    gefunden_list = cursor.fetchall()
    
    relevant_sql = 'SELECT distinct ID FROM "SYSTEM"."T3N" where CATEGORY = \'Marketing\' '
    cursor.execute(relevant_sql)
    relevant_list = cursor.fetchall()
    
    schnitt = 'SELECT * FROM ('+relevant_sql+') as t1, ('+gefunden_sql+') as t2 where t1.ID = t2.ID'
    cursor.execute(schnitt)
    schnitt_list = cursor.fetchall()
    
    recall = abs((len(schnitt_list)) / len(relevant_list))
    precision = abs((len(schnitt_list)) / len(gefunden_list))

    print(f"Recall: {recall} ")
    print(f"Precision: {precision} ")

In [25]:
print("Facebook")
precisionRecall('\'Facebook\'' )

# TODO Plots

Facebook
Recall: 0.4375 
Precision: 0.2222222222222222 


## 2. Wie interpretieren Sie die Retrieval-Ergebnisse und die Plots (für Ihre eigenen Anfragen und mindestens für die Anfrage {'AIR': 2, 'BAG': 2, 'DEPLOYMENT':2, 'ACCIDENT':2} an NHTSA?

In [26]:
def precisionRecall(param):
    gefunden_sql = 'SELECT distinct CMPLID FROM "SYSTEM"."$TA_CDESCRIND" WHERE TA_TOKEN = ('+param+')'    
    cursor.execute(gefunden_sql)
    gefunden_list = cursor.fetchall()
    
    relevant_sql = 'SELECT distinct CMPLID FROM "SYSTEM"."CMPL100K" where COMPDESC = \'AIR BAGS\' '
    cursor.execute(relevant_sql)
    relevant_list = cursor.fetchall()
    
    schnitt = 'SELECT * FROM ('+relevant_sql+') as t1, ('+gefunden_sql+') as t2 where t1.CMPLID = t2.CMPLID'
    cursor.execute(schnitt)
    schnitt_list = cursor.fetchall()
    
    recall = abs((len(schnitt_list)) / len(relevant_list))
    precision = abs((len(schnitt_list)) / len(gefunden_list))

    print(f"Recall: {recall} ")
    print(f"Precision: {precision} ")

In [27]:
print("AIR")
precisionRecall('\'AIR\'' )
print("BAG")
precisionRecall('\'BAG\'' )
print("ACCIDENT")
precisionRecall('\'ACCIDENT\'' )
print("DEPLOYMENT")
precisionRecall('\'DEPLOYMENT\'' )

# TODO Plots

AIR
Recall: 0.5510590015128594 
Precision: 0.19617611417799918 
BAG
Recall: 0.2866868381240545 
Precision: 0.44457478005865103 
ACCIDENT
Recall: 0.14826021180030258 
Precision: 0.06486844282641073 
DEPLOYMENT
Recall: 0.0416036308623298 
Precision: 0.41044776119402987 
